# Klavis Local Sandbox

This notebook demonstrates the **Local Sandbox** lifecycle:
- **Acquire** a local sandbox VM with multiple MCP servers
- **Initialize** by uploading a `tar.gz` archive of workspace files
- **Dump** (export) the sandbox state as a downloadable `tar.gz`
- **Release** the sandbox

> **Note:** The sandbox shown in the outputs of this notebook has already been released and can no longer be reused. All sandbox IDs, URLs, and tokens in the output are expired and non-functional — there is no sensitive data.

## 1. Setup

In [1]:
import io
import os
import tarfile

import httpx
from dotenv import load_dotenv

load_dotenv()

BASE_URL = "https://api.klavis.ai"
HEADERS = {"Authorization": f"Bearer {os.environ['KLAVIS_API_KEY']}"}

## 2. Acquire a Local Sandbox

`POST /local-sandbox` — Spin up a VM with the requested MCP servers.

In [2]:
async with httpx.AsyncClient() as client:
    resp = await client.post(
        f"{BASE_URL}/local-sandbox",
        headers=HEADERS,
        json={"server_names": ["filesystem", "git", "terminal"]},
    )

acquire_data = resp.json()
SANDBOX_ID = acquire_data["local_sandbox_id"]
print(acquire_data)

{'local_sandbox_id': 'c0a830f9-d92c-4051-ba31-27782f3e9334', 'status': 'occupied', 'benchmark': None, 'servers': [{'server_name': 'filesystem', 'id': 'f823cf93-5185-4cbc-ab22-acf983898f69', 'mcp_server_url': 'https://local-dev.klavis.ai/filesystem/mcp/?local_sandbox_id=c0a830f9-d92c-4051-ba31-27782f3e9334', 'status': 'occupied', 'benchmark': None, 'updated_at': '2026-02-16T12:45:25.770231-08:00', 'metadata': None, 'auth_data': None, 'tags': None}, {'server_name': 'git', 'id': '9bc2a172-cde4-4e0e-b748-ade7bdacad38', 'mcp_server_url': 'https://local-dev.klavis.ai/git/mcp/?local_sandbox_id=c0a830f9-d92c-4051-ba31-27782f3e9334', 'status': 'occupied', 'benchmark': None, 'updated_at': '2026-02-16T12:45:25.770231-08:00', 'metadata': None, 'auth_data': None, 'tags': None}, {'server_name': 'terminal', 'id': '860ecd8b-68ef-4e17-afd2-787f75c67e42', 'mcp_server_url': 'https://local-dev.klavis.ai/terminal/mcp/?local_sandbox_id=c0a830f9-d92c-4051-ba31-27782f3e9334', 'status': 'occupied', 'benchmark'

## 3. Initialize — Upload Data into the Sandbox

Three-step process:
1. `POST /local-sandbox/{id}/upload-url` — get a signed upload URL
2. `PUT` the `tar.gz` archive to that URL
3. `POST /local-sandbox/{id}/initialize` — extract the archive in the sandbox

In [3]:
# Create sample files and pack into tar.gz
files = {
    "hello.txt": "Hello from Klavis Local Sandbox!",
    "notes.md": "# Notes\n- item 1\n- item 2",
    "config.json": '{"app": "demo", "version": "1.0"}',
}

buf = io.BytesIO()
with tarfile.open(fileobj=buf, mode="w:gz") as tar:
    for name, content in files.items():
        data = content.encode()
        info = tarfile.TarInfo(name=name)
        info.size = len(data)
        tar.addfile(info, io.BytesIO(data))
buf.seek(0)

print(f"Archive: {buf.getbuffer().nbytes} bytes, {len(files)} files")

Archive: 221 bytes, 3 files


In [4]:
# Step 1: Get signed upload URL
async with httpx.AsyncClient() as client:
    resp = await client.post(
        f"{BASE_URL}/local-sandbox/{SANDBOX_ID}/upload-url",
        headers=HEADERS,
    )
upload_url = resp.json()["upload_url"]
print(f"Upload URL obtained (expires in {resp.json()['expires_in_minutes']} min)")

# Step 2: Upload tar.gz to signed URL
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.put(
        upload_url,
        headers={"Content-Type": "application/gzip"},
        content=buf.getvalue(),
    )
print(f"Upload status: {resp.status_code}")

# Step 3: Initialize (extract archive in sandbox)
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/local-sandbox/{SANDBOX_ID}/initialize",
        headers=HEADERS,
    )
print(resp.json())

Upload URL obtained (expires in 30 min)
Upload status: 200
{'sandbox_id': 'c0a830f9-d92c-4051-ba31-27782f3e9334', 'status': 'occupied', 'message': 'Extracted archive from GCS to workspace'}


## 4. Dump — Export Sandbox Data

1. `GET /local-sandbox/{id}/dump` — get a signed download URL
2. `GET` the signed URL to download the `tar.gz` archive

In [5]:
# Step 1: Get signed download URL
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.get(
        f"{BASE_URL}/local-sandbox/{SANDBOX_ID}/dump",
        headers=HEADERS,
    )
dump_info = resp.json()
print(f"Download URL obtained (expires in {dump_info['expires_in_minutes']} min)")

# Step 2: Download and inspect the archive
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.get(dump_info["download_url"])

with tarfile.open(fileobj=io.BytesIO(resp.content), mode="r:gz") as tar:
    print("Files in sandbox:")
    for m in tar.getmembers():
        print(f"  {m.name} ({m.size} bytes)")

Download URL obtained (expires in 30 min)
Files in sandbox:
  . (0 bytes)
  ./config.json (33 bytes)
  ./hello.txt (32 bytes)
  ./notes.md (25 bytes)


## 5. Release the Sandbox

`DELETE /local-sandbox/{id}` — Release the VM and clean up.

In [6]:
async with httpx.AsyncClient() as client:
    resp = await client.delete(
        f"{BASE_URL}/local-sandbox/{SANDBOX_ID}",
        headers=HEADERS,
    )
print(resp.json())

{'message': 'Local sandbox release started for c0a830f9-d92c-4051-ba31-27782f3e9334: it will be released asynchronously in background.'}
